In [79]:
from google.cloud import storage
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/weiqiang/.dotbot/cloud/quakeflow_wayne.json"

def download_blob(bucket_name, source_blob_name, destination_file_name):

    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)

    blob = bucket.blob(source_blob_name)
    blob.download_to_filename(destination_file_name)

    print("Blob {} downloaded to {}.".format(source_blob_name, destination_file_name))

In [80]:
download_blob("ncedc", "catalogs/combined_phases.csv", "combined_phases.csv")

Blob catalogs/combined_phases.csv downloaded to combined_phases.csv.


In [81]:
import pandas as pd
import h5py

In [82]:
catalogs = pd.read_csv("combined_phases.csv", sep="\t", dtype=str)

In [83]:
select_catalog = catalogs.sample(10, random_state=123)
select_catalog.to_csv("selected_phases.csv", sep="\t")

In [84]:
if not os.path.exists("data"):
    os.mkdir("data")
for fname in select_catalog["fname"]:
    download_blob("ncedc", f"data/{fname}", f"data/{fname}")

Blob data/NC.PHOB..HN.0171690.npz downloaded to data/NC.PHOB..HN.0171690.npz.
Blob data/TA.O02D..BH.0327980.npz downloaded to data/TA.O02D..BH.0327980.npz.
Blob data/NP.1865..HN.0345365.npz downloaded to data/NP.1865..HN.0345365.npz.
Blob data/NC.CDV..EH.0047896.npz downloaded to data/NC.CDV..EH.0047896.npz.
Blob data/BG.NEG..DP.0274080.npz downloaded to data/BG.NEG..DP.0274080.npz.
Blob data/NC.KMP..EH.0085448.npz downloaded to data/NC.KMP..EH.0085448.npz.
Blob data/NC.BPF..EH.0080915.npz downloaded to data/NC.BPF..EH.0080915.npz.
Blob data/NC.GAXB..EH.0390104.npz downloaded to data/NC.GAXB..EH.0390104.npz.
Blob data/BG.AL2..DP.0250724.npz downloaded to data/BG.AL2..DP.0250724.npz.
Blob data/NC.MCB..HH.0409320.npz downloaded to data/NC.MCB..HH.0409320.npz.


In [86]:
select_catalog.to_hdf('data.h5', '/catalog', mode='w', format='table', data_columns=True)
with h5py.File("data.h5", "a", libver='latest') as fp:
#     catalog = fp.create_group("/catalog")
    data = fp.create_group("/data")
    for fname in select_catalog["fname"]:
        meta = np.load(f"data/{fname}")
        ds = data.create_dataset(fname, data=meta["data"], dtype="float32")
        for k in meta:
            if k != "data":
                if meta[k].dtype.type is np.str_:
                    ds.attrs[k] = str(meta[k])
                else:
                    ds.attrs[k] = meta[k]

In [95]:
for fname in select_catalog["fname"]:
    meta = np.load(f"data/{fname}")
    for k in meta:
        print(k, meta[k], meta[k].dtype,len(meta[k]), hasattr(meta[k], "__len__"))
    break

data [[ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 1.5668182e-05  9.1718714e-05 -4.7859554e-05]
 [ 4.1721720e-05  5.2632578e-05 -2.2651691e-06]
 ...
 [ 6.8917012e-05  9.1712856e-05 -6.0328544e-05]
 [ 1.0286275e-05 -6.0008560e-06 -1.7058085e-06]
 [ 6.8910689e-05  3.9597282e-05  2.4346051e-05]] float32 12000 True


TypeError: len() of unsized object

In [90]:
with h5py.File("data.h5", "r", libver='latest', swmr=True) as fp:
    print(list(fp['data'].keys()))
    for k in fp['data'].keys():
        print(fp[f"data/{k}"].attrs["snr"])

['BG.AL2..DP.0250724.npz', 'BG.NEG..DP.0274080.npz', 'NC.BPF..EH.0080915.npz', 'NC.CDV..EH.0047896.npz', 'NC.GAXB..EH.0390104.npz', 'NC.KMP..EH.0085448.npz', 'NC.MCB..HH.0409320.npz', 'NC.PHOB..HN.0171690.npz', 'NP.1865..HN.0345365.npz', 'TA.O02D..BH.0327980.npz']
[ 7.3553972   8.04529806 12.2074562 ]
[ 61.4208811  116.28012702  44.25649176]
[ 1.          1.         78.90010683]
[ 1.          1.         29.35679672]
[  1.          1.        702.0602767]
[ 1.          1.         19.47826633]
[4.29846949 4.66984617 5.89343694]
[6.24967977 4.1921592  5.64781842]
[32.8693813  13.56657089 61.50129117]
[3.0837644  2.3433137  6.59059096]


In [91]:
pd.read_hdf("data.h5", "/catalog")

,fname,network,station,location_code,p_idx,p_time,p_remark,p_weight,s_idx,s_time,...,emergence_angle,azimuth,station_latitude,station_longitude,station_elevation_m,unit,dt,event_index,channels,snr
184748,NC.PHOB..HN.0171690.npz,NC,PHOB,--,6001,2004-10-07T20:50:40.560,IP,0,6178,2004-10-07T20:50:42.330,...,118.0,278.0,35.8666,-120.4796,796.0,m/s**2,0.01,171690,"HNE,HNN,HNZ","6.25,4.19,5.65"
889701,TA.O02D..BH.0327980.npz,TA,O02D,--,6000,2015-04-07T06:47:25.150,IP,1,6378,2015-04-07T06:47:28.920,...,92.0,12.0,40.1766,-122.7884,965.0,m/s,0.01,327980,"BHE,BHN,BHZ","3.08,2.34,6.59"
996432,NP.1865..HN.0345365.npz,NP,1865,--,6000,2016-07-18T09:27:29.990,IP,0,6396,2016-07-18T09:27:33.940,...,56.0,226.0,36.8831,-121.3908,74.0,m/s**2,0.01,345365,"HNE,HNN,HNZ","32.87,13.57,61.50"
1386146,NC.CDV..EH.0047896.npz,NC,CDV,--,6000,1986-03-17T04:25:09.940,IP,1,6362,1986-03-17T04:25:13.550,...,102.0,83.0,37.5665,-121.6793,262.0,m/s,0.01,47896,EHZ,"1.00,1.00,29.36"
588614,BG.NEG..DP.0274080.npz,BG,NEG,--,6000,2012-04-05T17:46:33.290,IP,1,6075,2012-04-05T17:46:34.030,...,118.0,97.0,38.8305,-122.7699,922.0,m/s,0.01,274080,"DPE,DPN,DPZ","61.42,116.28,44.26"
1481466,NC.KMP..EH.0085448.npz,NC,KMP,--,6000,1992-05-24T05:42:11.710,P,0,6373,1992-05-24T05:42:15.430,...,130.0,124.0,40.4175,-124.1206,931.0,m/s,0.01,85448,EHZ,"1.00,1.00,19.48"
1468963,NC.BPF..EH.0080915.npz,NC,BPF,--,6000,1991-09-11T01:30:26.000,EP,0,7219,1991-09-11T01:30:38.180,...,68.0,170.0,36.2303,-121.7720,349.0,m/s,0.01,80915,EHZ,"1.00,1.00,78.90"
1225679,NC.GAXB..EH.0390104.npz,NC,GAXB,--,6000,2019-05-29T19:14:38.270,IP,0,6187,2019-05-29T19:14:40.130,...,93.0,154.0,38.7107,-122.7573,386.0,m/s,0.01,390104,EHZ,"1.00,1.00,702.06"
456551,BG.AL2..DP.0250724.npz,BG,AL2,--,6000,2010-11-01T10:04:00.270,EP,2,6216,2010-11-01T10:04:02.420,...,66.0,278.0,38.8160,-122.8980,657.0,m/s,0.01,250724,"DPE,DPN,DPZ","7.36,8.05,12.21"
1339241,NC.MCB..HH.0409320.npz,NC,MCB,--,6000,2020-07-29T19:52:48.150,IP,1,6211,2020-07-29T19:52:50.250,...,114.0,333.0,37.6444,-118.8968,2391.0,m/s,0.01,409320,"HHE,HHN,HHZ","4.30,4.67,5.89"
